In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import json, re, os, math, torch
from pathlib import Path
from tqdm import tqdm

BASE_DIR = Path("/content/drive/MyDrive/HotpotQA_snapshot")
SRC_JSON = Path("/content/evidence_hotpotqa.json")                 
DST_JSON = BASE_DIR / "evidence_hotpotqa_entities.json"         

assert SRC_JSON.exists(), f"{SRC_JSON} not found!"


In [3]:
with SRC_JSON.open(encoding="utf-8") as f:
    data = json.load(f)

print(f"Total examples loaded: {len(data):,}")

MINI_KEYS = {"type", "question", "answer", "supports"}   

new_data = []
for item in data:
    pruned = {k: item[k] for k in MINI_KEYS if k in item}
    pruned["entities"] = []                              
    new_data.append(pruned)

print("Sample after pruning:\n", json.dumps(new_data[0], indent=2, ensure_ascii=False))


Total examples loaded: 100
Sample after pruning:
 {
  "question": "Were both Monkey Kingdom and Anaganaga O Dheerudu connected with Disney?",
  "type": "comparison",
  "answer": "yes",
  "supports": [
    [
      "Anaganaga O Dheerudu",
      " Co-produced by Disney World Cinema with veteran director K. Raghavendra Rao at a budget of 270 million, it stars Siddharth, Shruti Haasan, in her Telugu debut, and Harshitha as protagonists with Lakshmi Manchu making her debut in a negative role."
    ],
    [
      "Monkey Kingdom",
      " The film was released by Disneynature on April 17, 2015."
    ]
  ],
  "entities": []
}


In [4]:
!pip install -q -U "transformers>=4.51.0" accelerate bitsandbytes

from transformers import AutoModelForCausalLM, AutoTokenizer

MODEL_NAME = "Qwen/Qwen3-14B"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype="auto",
    device_map="auto"    # روی A100 به‌طور خودکار توزیع می‌شود
)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 117.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

model-00005-of-00008.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00004-of-00008.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/1.91G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [5]:
SYS_PROMPT = (
    "You are an expert information-retrieval assistant working with a large "
    "heterogeneous knowledge graph built from thousands of text chunks. "
    "Given a user QUESTION, your task is to identify **between 2 and 5 surface-form "
    "ENTITIES** (named entities or key noun-phrases) whose presence in the graph "
    "is most likely to lead an intelligent traversal agent toward the chunks "
    "needed to answer the question. "
    "Guidelines:\n"
    " Prioritise specific names (people, places, works, organisations) over generic terms.\n"
    " Prefer entities that can be disambiguated in isolation (e.g., “Python (programming language)” "
    "instead of just “Python” if context requires).\n"
    " Do **NOT** include explanatory text, reasoning, or duplicate entities.\n"
    " Return the result strictly as a JSON list of strings.\n\n"
    "QUESTION: {question}\n"
    "Your JSON:"
)

GEN_KWARGS = dict(
    max_new_tokens = 32768,
    temperature    = 0.6,
    top_p          = 0.95,
    top_k          = 20,
    do_sample      = True,
    eos_token_id   = tokenizer.eos_token_id,
)

def extract_entities(question: str) -> list[str]:
    """Run Qwen3-14B (thinking mode) and parse the JSON list of entities."""
    messages = [{"role": "user", "content": SYS_PROMPT.format(question=question)}]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=True            
    )
    inputs = tokenizer([text], return_tensors="pt").to(model.device)

    gen_ids = model.generate(**inputs, **GEN_KWARGS)
    out_ids = gen_ids[0][len(inputs.input_ids[0]):].tolist()

    try:
        end_think = len(out_ids) - out_ids[::-1].index(151668)  # </think>
    except ValueError:
        end_think = 0

    content_ids = out_ids[end_think:]
    content = tokenizer.decode(content_ids, skip_special_tokens=True).strip()

    try:
        json_str = re.search(r"\[.*\]", content, re.S).group(0)
        entities = json.loads(json_str)
        if isinstance(entities, list):
            return [e.strip() for e in entities if isinstance(e, str) and e.strip()]
    except Exception as e:
        print("⚠️ parse error:", e, "| raw content →", content)

    return []  #fallback


In [6]:
for item in tqdm(new_data, desc="Generating entities"):
    q = item["question"]
    ents = extract_entities(q)
    item["entities"] = ents if ents else None 

print("\nExample with entities:\n", json.dumps(new_data[0], indent=2, ensure_ascii=False))


Generating entities: 100%|██████████| 100/100 [1:18:13<00:00, 46.94s/it]


Example with entities:
 {
  "question": "Were both Monkey Kingdom and Anaganaga O Dheerudu connected with Disney?",
  "type": "comparison",
  "answer": "yes",
  "supports": [
    [
      "Anaganaga O Dheerudu",
      " Co-produced by Disney World Cinema with veteran director K. Raghavendra Rao at a budget of 270 million, it stars Siddharth, Shruti Haasan, in her Telugu debut, and Harshitha as protagonists with Lakshmi Manchu making her debut in a negative role."
    ],
    [
      "Monkey Kingdom",
      " The film was released by Disneynature on April 17, 2015."
    ]
  ],
  "entities": [
    "Monkey Kingdom",
    "Anaganaga O Dheerudu"
  ]
}


In [7]:
with DST_JSON.open("w", encoding="utf-8") as f:
    json.dump(new_data, f, ensure_ascii=False, indent=2)

print(f"✅  Saved → {DST_JSON}")


✅  Saved → /content/drive/MyDrive/HotpotQA_snapshot/evidence_hotpotqa_entities.json
